# 🎨 SDXL Vintage Illustration Notebook — PRO

Stable Diffusion XL / Turbo / SD 1.5 preconfigured for **vintage ink+watercolor** illustrations (old book style, retro cartoon aesthetic).

### Included
- **Mode Selector** (CPU / GPU basic / GPU optimal via `CONFIG`)
- **Gallery Manager (Flask web UI)** + **Logs** page
- **Text2Img** (default vintage style + custom scene prompt)
- **Img2Img** (photo → vintage illustration)
- **ControlNet (Canny)** for pose/contour consistency
- **Upscale** (x4) for higher resolution
- **Color tone control** (dropdown presets + custom override)
- Saving images + JSON metadata to `/content/outputs` (timestamped)



In [ ]:
## ✅ Tips

- Change **`color_tone`** or **`custom_tone`** any time, then regenerate.
- Use **`seed=None`** for randomization, or set a fixed integer for repeatability.
- For **Img2Img**, tweak **`strength`**:
  - lower = closer to original,
  - higher = stronger style.
- **ControlNet (Canny)** retains silhouette/contours. For different looks, adjust **`canny_low`/`canny_high`**.
- If memory errors occur, run **`free_memory()`** and re-run only the needed loader.


In [ ]:
#@title ⚙ Vars
REPO_NAME   = "sd-colab-gallery"
ORIGIN_URL  = f"https://github.com/tekswirl25/{REPO_NAME}.git"
repo_dir    = f"/content/{REPO_NAME}"
OUTPUT_DIR  = "/content/outputs"
LOG_DIR     = "/content/logs"


In [ ]:
# @title 🚀 Repo init
import os, sys
if not os.path.exists(repo_dir):
    !git clone $ORIGIN_URL
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

from scripts.repo_init import init_repo
repo_dir = init_repo(REPO_NAME, ORIGIN_URL, "")




In [ ]:
#@title Config Init
# -----------------------------
# UI-параметры (выборы перед запуском)
PROGRAM_VERSION = "SD15"       #@param ["SDXL", "SDXL_TURBO", "SD15"]
MODE            = "GPU_OPTIMAL" #@param ["GPU_OPTIMAL", "GPU_BASIC", "CPU"]
OUTPUT_DIR      = "/content/outputs"
# -----------------------------

# HuggingFace token (Colab Secrets или вручную)
hf_token = None
try:
    from google.colab import userdata
    hf_token = userdata.get("HF_TOKEN")
except Exception:
    pass

import torch
from scripts.config import init_config
from scripts.logger import log_info, log_error

CONFIG, VARIANT, DEFAULTS, AUTO_UPSCALE = init_config(
    model_variant=PROGRAM_VERSION,
    output_dir=OUTPUT_DIR,
    hf_token=hf_token,
    mode=MODE
)

log_info(f"Torch version: {torch.__version__}")
log_info(f"Device: {CONFIG['DEVICE']} | DType: {CONFIG['DTYPE']} | Mode: {CONFIG['MODE']}")


In [ ]:
# @title 📦 3 Install dependencies (final)
import os

if MODE in ["GPU_OPTIMAL", "GPU_BASIC"]:
    # GPU-варианты: Colab сам подберёт подходящий билд Torch под CUDA
    !pip install --quiet torch torchvision torchaudio
else:
    # CPU-режим
    !pip install --quiet torch torchvision torchaudio

# HuggingFace стек + Gradio
!pip install --quiet --upgrade diffusers transformers accelerate safetensors xformers gradio

# Общие пакеты
!pip install --quiet opencv-python-headless pillow ipywidgets

print(f"✅ Dependencies installed for MODE={MODE}")



In [ ]:
# @title 🌐 Start Gradio server (Logs + Gallery)
refresh_interval = 5  #@param {type:"integer"}
LOG_LINES = 50        #@param {type:"integer"}

from scripts.server_gradio import start_gradio_server

server = start_gradio_server("/content/outputs", refresh_interval=refresh_interval, LOG_LINES=LOG_LINES)

if server and hasattr(server, "share_url") and server.share_url:
    print(f"✅ Gradio server running at: {server.share_url}")
else:
    print("⚠️ Gradio server started, but no public URL detected")




In [ ]:
# @title 🔍 1.1 Тест MODE
from scripts.config import init_config

# CPU режим
config_cpu, *_ = init_config(model_variant="SDXL", mode="CPU")
print("CPU test:", config_cpu["DEVICE"], config_cpu["DTYPE"])

# GPU базовый
config_gpu_basic, *_ = init_config(model_variant="SDXL", mode="GPU_BASIC")
print("GPU_BASIC test:", config_gpu_basic["DEVICE"], config_gpu_basic["DTYPE"])

# GPU оптимальный
config_gpu_opt, *_ = init_config(model_variant="SDXL", mode="GPU_OPTIMAL")
print("GPU_OPTIMAL test:", config_gpu_opt["DEVICE"], config_gpu_opt["DTYPE"])



In [ ]:
# @title 🔁 4 Imports & utils
from scripts.utils import (
    ts_now, base_name, save_image_and_meta,
    free_memory, list_images, canny_from_image
)
from scripts.logger import log_info, log_error
log_info(f"Utils loaded. OUTPUT_DIR={CONFIG['OUTPUT_DIR']}")



In [ ]:
# @title 🧠 5 Model loaders
from scripts.loaders import (
    get_txt2img_pipe,
    get_img2img_pipe,
    get_controlnet_pipe,
    get_upscale_pipe,
    reset_pipes
)
log_info(f"Loader functions ready for variant: {CONFIG['MODEL_VARIANT']}")




In [ ]:
# @title 🎨 6 Style base & Prompt builder
from scripts.prompt_builder import build_style
from scripts.logger import log_info

base_style  = "illustration"  #@param ["illustration","photoreal","anime"]
color_tone  = "vintage"       #@param ["warm","cool","vintage"]
negative    = ""              #@param {type:"string"}

# строим style_suffix (глобально для всех модулей)
style_suffix = build_style(style=base_style, tone=color_tone)

log_info(f"Style ready | base={base_style}, tone={color_tone}, suffix={style_suffix}")


In [ ]:
# @title 🖼 Text2Img
from scripts.config import CONFIG, DEVICE, DTYPE, VARIANT_MODELS, DEFAULTS
from scripts.loaders import get_txt2img_pipe
from scripts.utils import save_image_and_meta, ts_now
from scripts.logger import log_info
import torch

# user_prompt вынесен сюда
user_prompt = "A thoughtful man, slightly resembling Donald Trump."  #@param {type:"string"}

seed = 12345   #@param {type:"number"}  # 0 → авторандом
n    = 1   #@param {type:"number"}

# финальный промпт формируется только здесь
final_prompt = f"{user_prompt}, {style_suffix}" if style_suffix else user_prompt

variant  = CONFIG["MODEL_VARIANT"]
model_id = VARIANT_MODELS[variant]["txt2img"]
pipe     = get_txt2img_pipe(model_id, DEVICE, DTYPE)

steps     = DEFAULTS["txt2img_steps"]
cfg_scale = DEFAULTS["txt2img_cfg"]
height, width = DEFAULTS["img_size"]

# сид — фиксированный или авто + инкремент для нескольких картинок
base_seed = int(seed) if seed and int(seed) > 0 else int(torch.seed())

saved = []
for i in range(int(n)):
    current_seed = base_seed + i
    generator = torch.manual_seed(current_seed)

    out = pipe(
        prompt=final_prompt,
        negative_prompt=(negative or None),
        height=height, width=width,
        guidance_scale=cfg_scale,
        num_inference_steps=steps,
        generator=generator,
    )
    im = out.images[0]
    meta = {
        "mode": "text2img",
        "prompt": final_prompt,
        "negative": negative,
        "steps": steps,
        "cfg_scale": cfg_scale,
        "size": [height, width],
        "seed": current_seed,
        "timestamp": ts_now(),
    }
    p, _ = save_image_and_meta(
        im,
        prefix="text2img",
        meta=meta,
        output_dir=CONFIG["OUTPUT_DIR"],
        mode="text2img"
    )
    saved.append(p)

log_info(f"Text2Img saved: {saved}")


In [ ]:
# @title 📤 Upload image (опционально для Img2Img / ControlNet)
from google.colab import files
import os

# Можно заранее указать путь вручную:
src_path = ""  #@param {type:"string"}

uploaded = files.upload()  # откроет окно "Browse"

if uploaded:
    filename = list(uploaded.keys())[0]
    src_path = os.path.join("/content", filename)
    file_size = os.path.getsize(src_path) / 1024
    print(f"✅ Uploaded: {filename}")
    print(f"📂 Saved to: {src_path} ({file_size:.1f} KB)")
elif src_path:
    print(f"⚠️ No upload. Использую вручную заданный путь: {src_path}")
else:
    print("⚠️ No file uploaded and src_path пустой. Укажи источник изображения.")


In [ ]:
# @title 🖼 Img2Img — unified save & metadata (thumbnails, JSON, folders)
from PIL import Image
import torch
from scripts.prompt_builder import build_prompt
from scripts.loaders import get_img2img_pipe
from scripts.utils import save_image_and_meta, ts_now
from scripts.config import VARIANT_MODELS

# UI
strength = 0.6   #@param {type:"number"}
seed     = 12345 #@param {type:"number"}
src_path = ""    #@param {type:"string"}
user_prompt = ""  #@param {type:"string"}


# 1) Build final prompt (same logic as Text2Img)
final_prompt = build_prompt(
    text=user_prompt,
    style=base_style,
    tone=color_tone
)

# 2) Prepare pipeline
variant  = CONFIG["MODEL_VARIANT"]
model_id = VARIANT_MODELS[variant]["img2img"]

image = Image.open(src_path).convert("RGB")
pipe_i2i = get_img2img_pipe(model_id, CONFIG["DEVICE"], CONFIG["DTYPE"])
generator = torch.manual_seed(seed)

# 3) Inference
out = pipe_i2i(
    prompt=final_prompt,
    negative_prompt=negative or None,
    image=image,
    strength=strength,
    num_inference_steps=DEFAULTS["img2img_steps"],
    guidance_scale=DEFAULTS["img2img_cfg"],
    generator=generator,
)

im = out.images[0]

# 4) Save with thumbnails & JSON in proper subfolder
meta = {
    "mode": "img2img",
    "prompt": final_prompt,
    "negative": negative,
    "steps": DEFAULTS["img2img_steps"],
    "cfg_scale": DEFAULTS["img2img_cfg"],
    "strength": strength,
    "seed": seed,
    "source_path": src_path,
    "timestamp": ts_now(),
}
img_path, meta_path = save_image_and_meta(
    im=im,
    prefix="img2img",
    meta=meta,
    output_dir=CONFIG["OUTPUT_DIR"],
    mode="img2img",             # 👈 ensures /outputs/img2img and /thumbnails/img2img
)

print("✅ Img2Img completed.")
print("📂 Saved:", img_path)
print("🧾 Meta:", meta_path)


In [ ]:
# @title 📤 Upload image (опционально для ControlNet)
from google.colab import files
import os

uploaded = files.upload()  # откроет окно "Browse"

if uploaded:
    filename = list(uploaded.keys())[0]
    control_path = os.path.join("/content", filename)
    file_size = os.path.getsize(control_path) / 1024
    print(f"✅ Uploaded: {filename}")
    print(f"📂 Saved to: {control_path} ({file_size:.1f} KB)")
else:
    print("⚠️ No file uploaded. Используй control_path вручную в следующей ячейке.")



In [ ]:
# @title 🧭 ControlNet — unified save & metadata (thumbnails, JSON, folders)
from PIL import Image
import torch
from scripts.prompt_builder import build_prompt
from scripts.loaders import get_controlnet_pipe
from scripts.utils import save_image_and_meta, ts_now, canny_from_image
from scripts.config import VARIANT_MODELS

control_path = "" #@param {type:"string"}
low_thr      = 100   #@param {type:"number"}
high_thr     = 200   #@param {type:"number"}
seed         = 12345 #@param {type:"number"}
user_prompt = ""  #@param {type:"string"}

# 1) Build final prompt (same logic as Text2Img)
final_prompt = build_prompt(
    text=user_prompt,
    style=base_style,
    tone=color_tone
)

# 2) Prepare pipeline (handle base/control ids across variants)
variant         = CONFIG["MODEL_VARIANT"]
controlnet_id   = VARIANT_MODELS[variant]["controlnet"]
base_id         = VARIANT_MODELS[variant].get("controlnet_model", VARIANT_MODELS[variant]["txt2img"])

pipe_cn = get_controlnet_pipe(
    model_id=base_id,
    controlnet_id=controlnet_id,
    device=CONFIG["DEVICE"],
    dtype=CONFIG["DTYPE"],
)

# 3) Source and (optional) Canny preprocessor as control image
src_img = Image.open(control_path).convert("RGB")
control_img = canny_from_image(src_img, low=low_thr, high=high_thr)

generator = torch.manual_seed(seed)

# NOTE:
# diffusers SDXL ControlNet expects `image` (init image for img2img-like conditioning is optional)
# and `control_image` for the canny map. We pass both for robustness.
out = pipe_cn(
    prompt=final_prompt,
    negative_prompt=negative or None,
    image=src_img,
    control_image=control_img,
    num_inference_steps=DEFAULTS["controlnet_steps"],
    guidance_scale=DEFAULTS["controlnet_cfg"],
    generator=generator,
)

im = out.images[0]

# 4) Save with thumbnails & JSON in proper subfolder
meta = {
    "mode": "controlnet",
    "prompt": final_prompt,
    "negative": negative,
    "steps": DEFAULTS["controlnet_steps"],
    "cfg_scale": DEFAULTS["controlnet_cfg"],
    "seed": seed,
    "source_path": control_path,
    "canny": {"low": low_thr, "high": high_thr},
    "timestamp": ts_now(),
}
img_path, meta_path = save_image_and_meta(
    im=im,
    prefix="controlnet",
    meta=meta,
    output_dir=CONFIG["OUTPUT_DIR"],
    mode="controlnet",          # 👈 ensures /outputs/controlnet and /thumbnails/controlnet
)

print("✅ ControlNet completed.")
print("📂 Saved:", img_path)
print("🧾 Meta:", meta_path)
print(f"⚙️ Thresholds: low={low_thr}, high={high_thr}")


In [ ]:
# @title ⬆️ 10 Upscale x4
from PIL import Image

variant = CONFIG["MODEL_VARIANT"]
up_id   = VARIANT_MODELS[variant]["upscale"]

pipe_up = get_upscale_pipe(up_id, DEVICE, DTYPE)

in_path = ""   #@param {type:"string"}
seed    = 12345 #@param {type:"number"}

if not in_path:
    raise ValueError("Нужно указать путь к изображению для апскейла (in_path)")

img = Image.open(in_path).convert("RGB")
generator = torch.manual_seed(seed)

out = pipe_up(image=img, prompt=final_prompt, generator=generator)

im = out.images[0]
meta = {
    "mode": "upscale_x4",
    "prompt": final_prompt,
    "seed": seed,
    "timestamp": ts_now(),
}
p, _ = save_image_and_meta(
    im,
    prefix="upscale",
    meta=meta,
    output_dir=CONFIG["OUTPUT_DIR"],
    mode="upscale"
)
log_info(f"Upscale saved: {p}")
